In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
elmo = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]

In [ ]:

# Provide input tensor and create embeddings
bad_review_text = 'George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. "Do we get to win this time?" Yes, you do.\n\nDid the reviewer find this movie good or bad? bad'
good_review_text = 'George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. "Do we get to win this time?" Yes, you do.\n\nDid the reviewer find this movie good or bad? good'

# input_tensor = ["my birthday is the best day of the year"]
input_tensor = [bad_review_text, good_review_text]
embeddings_tensor = elmo(tf.constant(input_tensor))
embeddings_tensor["lstm_outputs1"].shape

#, signature="default", as_dict=True)

# #with tf.Session() as sess:
#    # sess.run(tf.global_variables_initializer())
#     #embeddings = sess.run(embeddings_tensor)
# print(embeddings_tensor.shape)
# print(embeddings_tensor)


In [ ]:
import torch
torch.tensor(embeddings_tensor["elmo"].numpy().tolist()).shape

In [ ]:
torch.tensor(embeddings_tensor["elmo"].numpy()).mean(dim=1).shape

In [ ]:
from transformers import (
    PretrainedConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
)

class ElmoConfig(PretrainedConfig):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = 1024
        self.num_hidden_layers = 1
        self.is_encoder_decoder = False
        self.architectures = ["Elmo"]

class TfElmoTokenizer(PreTrainedTokenizer):

    def __call__(
        self, text, return_tensors, truncation, return_offsets_mapping, text_target
    ):
        return text

    @staticmethod
    def from_pretrained(path):
        return TfElmoTokenizer()


class TfElmoModel(PreTrainedModel):
    def __init__(self):
        super().__init__(config=ElmoConfig())
        self.elmo_model = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]

    def forward(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
    ):
        embeddings_tensor = self.elmo_model(tf.constant(input_ids))
        elmo_embeddings = embeddings_tensor["elmo"]
        return torch.tensor(elmo_embeddings.numpy())


    @staticmethod
    def from_pretrained(path):
        return TfElmoModel()

sequences = [
    "my birthday is the best day of the year: yes",
    "my birthday is the best day of the year: no",
]

tokenizer = TfElmoTokenizer.from_pretrained("")
model = TfElmoModel.from_pretrained("")
input = tokenizer(sequences, True, None, None, None)
model(input)

In [ ]:
model(input).shape

In [ ]:
torch.mean(model(input), dim=1).shape